<a href="https://colab.research.google.com/github/TuyetMaiLe/COVID19-Fake-News-Detection-in-English/blob/main/Lemmatization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download("stopwords")
nltk.download('wordnet')
!pip install emoji

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
     |████████████████████████████████| 51kB 5.7MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49715 sha256=1980c093467e17c0406b714dda2e023bedb3957dd0801c639f3d57ad80d3e742
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from nltk.tokenize import TweetTokenizer 
from emoji import demojize
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from nltk.stem import WordNetLemmatizer


In [ ]:
from sklearn.model_selection import GridSearchCV 

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

#Pre_Processing

In [ ]:
Dataset_train=pd.read_excel("/content/Constraint_English_Train.xlsx")
Dataset_test= pd.read_excel("/content/Constraint_English_Val.xlsx")
Data_train = Dataset_train.drop('id', 1)
Data_test = Dataset_test.drop('id', 1)

In [ ]:
Data_train_3=Dataset_train.drop('id', 1)
Data_train_4=Dataset_train.drop('id', 1)

In [ ]:
Data_test_3=Dataset_test.drop('id', 1)
Data_test_4=Dataset_test.drop('id', 1)

In [ ]:
tokenizer = TweetTokenizer()

In [ ]:
def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

In [ ]:
def normalizeTweet(tweet):
    tokens = tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = normTweet.replace("cannot ", "can not ").replace("n't ", " n't ").replace("n 't ", " n't ").replace("ca n't", "can't").replace("ai n't", "ain't")
    normTweet = normTweet.replace("'m ", " 'm ").replace("'re ", " 're ").replace("'s ", " 's ").replace("'ll ", " 'll ").replace("'d ", " 'd ").replace("'ve ", " 've ")
    normTweet = normTweet.replace(" p . m .", "  p.m.") .replace(" p . m ", " p.m ").replace(" a . m .", " a.m.").replace(" a . m ", " a.m ")

    normTweet = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normTweet)
    
    return " ".join(normTweet.split())

In [ ]:
Data_train.loc[:,"tweet"] = Data_train['tweet'].apply(lambda x: normalizeTweet(x))
Data_train.loc[:,"label"] = Data_train['label'].apply(lambda x: 1 if x == 'fake' else 0)
Data_train.columns = ["tweet", "labels"]

In [ ]:
Data_train_3.loc[:,"tweet"] = Data_train_3['tweet'].apply(lambda x: normalizeTweet(x))
Data_train_3.loc[:,"label"] = Data_train_3['label'].apply(lambda x: 1 if x == 'fake' else 0)
Data_train_3.columns = ["tweet", "labels"]

Data_train_4.loc[:,"tweet"] = Data_train_4['tweet'].apply(lambda x: normalizeTweet(x))
Data_train_4.loc[:,"label"] = Data_train_4['label'].apply(lambda x: 1 if x == 'fake' else 0)
Data_train_4.columns = ["tweet", "labels"]

In [ ]:
#test đã tiền xử lý
Data_test.loc[:,"tweet"] = Data_test['tweet'].apply(lambda x: normalizeTweet(x))
Data_test.loc[:,"label"] = Data_test['label'].apply(lambda x: 1 if x == 'fake' else 0)
Data_test.columns = ["tweet", "labels"]

In [ ]:
Data_test_3.loc[:,"tweet"] = Data_test_3['tweet'].apply(lambda x: normalizeTweet(x))
Data_test_3.loc[:,"label"] = Data_test_3['label'].apply(lambda x: 1 if x == 'fake' else 0)
Data_test_3.columns = ["tweet", "labels"]

Data_test_4.loc[:,"tweet"] = Data_test_4['tweet'].apply(lambda x: normalizeTweet(x))
Data_test_4.loc[:,"label"] = Data_test_4['label'].apply(lambda x: 1 if x == 'fake' else 0)
Data_test_4.columns = ["tweet", "labels"]

In [ ]:
Data_train = Data_train.apply(lambda X_train: X_train.astype(str).str.lower())
Data_test = Data_test.apply(lambda X_train: X_train.astype(str).str.lower())

In [ ]:
Data_train

,tweet,labels
0,the cdc currently reports 99031 deaths . in ge...,0
1,states reported 1121 deaths a small rise from ...,0
2,politically correct woman ( almost ) uses pand...,1
3,#indiafightscorona : we have 1524 #covid testi...,0
4,populous states can generate large case counts...,0
...,...,...
6415,a tiger tested positive for covid - 19 please ...,1
6416,? ? ? autopsies prove that covid - 19 is ? ? �...,1
6417,_a post claims a covid - 19 vaccine has alread...,1
6418,aamir khan donate 250 cr . in pm relief cares ...,1


** Train dữ liệu** + **Lemmatization** + **stopword**

In [ ]:
STOPWORDS = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def pre_Process_LS(text):
  text= " ".join([word for word in str(text).split() if word not in STOPWORDS])
  text= " ".join([lemmatizer.lemmatize(word) for word in text.split()])
  return text

In [ ]:
Data_train_3.loc[:,"tweet"] = Data_train_3["tweet"].apply(lambda X_train: pre_Process_LS(X_train))
Data_test_3.loc[:,"tweet"] = Data_test_3["tweet"].apply(lambda X_train: pre_Process_LS(X_train))

In [ ]:
Data_train_3 = Data_train_3.apply(lambda X_train: X_train.astype(str).str.lower())
Data_test_3 = Data_test_3.apply(lambda X_train: X_train.astype(str).str.lower())

In [ ]:
Data_train_3

,tweet,labels
0,cdc currently report 99031 death . general dis...,0
1,state reported 1121 death small rise last tues...,0
2,politically correct woman ( almost ) us pandem...,1
3,#indiafightscorona : 1524 #covid testing labor...,0
4,populous state generate large case count look ...,0
...,...,...
6415,tiger tested positive covid - 19 please stay a...,1
6416,? ? ? autopsy prove covid - 19 ? ? � blood clo...,1
6417,_a post claim covid - 19 vaccine already devel...,1
6418,aamir khan donate 250 cr . pm relief care fund,1


In [ ]:
X_train_3=Data_train_3.iloc[:,0].values
Y_train_3=Data_train_3.iloc[:,1].values

In [ ]:
X_test_3=Data_test_3.iloc[:,0].values
Y_test_3=Data_test_3.iloc[:,1].values

In [ ]:
TF_IDF_vectorizer = TfidfVectorizer()

In [ ]:
X_train_3 = TF_IDF_vectorizer.fit_transform(X_train_3).toarray()
X_test_3 = TF_IDF_vectorizer.transform(X_test_3).toarray()

In [ ]:
def Evaluation_metrics(Y_test, Y_Pred):
  print('accuracy ',accuracy_score(Y_test,Y_Pred))
  print('precision ', precision_score(Y_test,Y_Pred,average= 'weighted'))
  print('recall ', recall_score(Y_test,Y_Pred,average= 'weighted'))
  print("f1", f1_score(Y_test,Y_Pred, average= 'weighted'))
  #print(classification_report(Y_test, Y_Pred, target_names = ["FAKE","REAL"]))
  #confusion_matrix(Y_test,Y_Pred, labels=[0,1])
  cm = confusion_matrix(Y_test,Y_Pred)
  print(cm)

In [ ]:
kf=KFold(n_splits=5, random_state=0, shuffle=False)
for train_index, test_index in kf.split(X_train_3):
  print("TRAIN:", train_index, "TEST:", test_index)
  x_train, x_test = X_train_3[train_index], X_train_3[test_index]
  y_train, y_test = Y_train_3[train_index], Y_train_3[test_index]
  y_test = y_test.reshape(-1,1)
  y_train = y_train.reshape(-1,1)
  classsifier_NB =GaussianNB()
  classsifier_NB.fit(x_train, y_train)
  Y_pred_3 = classsifier_NB.predict(x_test)
  Y_pred_3 = Y_pred_3.reshape(-1,1)
  #cm = confusion_matrix(y_test,Y_pred_1)
  #print(cm)
  #print (Y_pred_1.shape)
  #print (y_test.shape)
  Evaluation_metrics(y_test, Y_pred_3)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


TRAIN: [1284 1285 1286 ... 6417 6418 6419] TEST: [   0    1    2 ... 1281 1282 1283]
accuracy  0.8247663551401869
precision  0.8300082830169653
recall  0.8247663551401869
f1 0.822898889228164
[[622  66]
 [159 437]]
TRAIN: [   0    1    2 ... 6417 6418 6419] TEST: [1284 1285 1286 ... 2565 2566 2567]


/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy  0.8489096573208723
precision  0.8500756571369819
recall  0.8489096573208723
f1 0.8482863963017276
[[615  73]
 [121 475]]
TRAIN: [   0    1    2 ... 6417 6418 6419] TEST: [2568 2569 2570 ... 3849 3850 3851]


/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy  0.8341121495327103
precision  0.840389887082625
recall  0.8341121495327103
f1 0.8331655248275007
[[588  62]
 [151 483]]
TRAIN: [   0    1    2 ... 6417 6418 6419] TEST: [3852 3853 3854 ... 5133 5134 5135]


/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy  0.8652647975077882
precision  0.8666223809193965
recall  0.8652647975077882
f1 0.8648685433994194
[[609  64]
 [109 502]]
TRAIN: [   0    1    2 ... 5133 5134 5135] TEST: [5136 5137 5138 ... 6417 6418 6419]


/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy  0.8255451713395638
precision  0.8305721709347125
recall  0.8255451713395638
f1 0.8244508857941436
[[591  70]
 [154 469]]


In [ ]:
Data_test_3

,tweet,labels
0,chinese converting islam realising muslim affe...,1
1,11 13 people ( diamond princess cruise ship ) ...,1
2,"covid - 19 caused bacterium , virus treated as...",1
3,mike penny rnc speech praise donald trump 's c...,1
4,6/10 sky 's @user explains latest #covid19 dat...,0
...,...,...
2135,donald trump wrongly claimed new zealand “ big...,1
2136,current understanding #covid19 spread mostly p...,0
2137,nothing scream “ sat around fuck lockdown ” qu...,1
2138,birx say covid - 19 outbreak control ‘ people ...,1


In [ ]:
y_pred = classsifier_NB.predict(X_test_3)

In [ ]:
y_pred = y_pred.reshape(-1,1)

In [ ]:
Y_test_3=Y_test_3.reshape(-1,1)

In [ ]:
print(y_pred.shape)

(2140, 1)


In [ ]:
print(Y_test_3.shape)

(2140, 1)


In [ ]:
Evaluation_metrics(Y_test_3, y_pred)

accuracy  0.8345794392523365
precision  0.8370398956083542
recall  0.8345794392523365
f1 0.8338060644196306
[[995 125]
 [229 791]]


Logistic

In [ ]:
kf=KFold(n_splits=5, random_state=0, shuffle=False)
for train_index, test_index in kf.split(X_train_3):
  print("TRAIN:", train_index, "TEST:", test_index)
  x_train, x_test = X_train_3[train_index], X_train_3[test_index]
  y_train, y_test = Y_train_3[train_index], Y_train_3[test_index]
  y_test = y_test.reshape(-1,1)
  y_train = y_train.reshape(-1,1)
  classifier_LR_Val =LogisticRegression()
  classifier_LR_Val.fit(x_train, y_train)
  Y_pred = classifier_LR_Val.predict(x_test)
  Y_pred = Y_pred.reshape(-1,1)
  cm = confusion_matrix(y_test,Y_pred)
  print(cm)
  print (Y_pred.shape)
  print (y_test.shape)
  Evaluation_metrics(y_test, Y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


TRAIN: [1284 1285 1286 ... 6417 6418 6419] TEST: [   0    1    2 ... 1281 1282 1283]
[[626  62]
 [ 27 569]]
(1284, 1)
(1284, 1)
accuracy  0.9306853582554517
precision  0.9322366190148182
recall  0.9306853582554517
f1 0.9307693999000298
[[626  62]
 [ 27 569]]
TRAIN: [   0    1    2 ... 6417 6418 6419] TEST: [1284 1285 1286 ... 2565 2566 2567]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[638  50]
 [ 47 549]]
(1284, 1)
(1284, 1)
accuracy  0.9244548286604362
precision  0.924489546687081
recall  0.9244548286604362
f1 0.9244671222732058
[[638  50]
 [ 47 549]]
TRAIN: [   0    1    2 ... 6417 6418 6419] TEST: [2568 2569 2570 ... 3849 3850 3851]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[601  49]
 [ 41 593]]
(1284, 1)
(1284, 1)
accuracy  0.9299065420560748
precision  0.9299841810541435
recall  0.9299065420560748
f1 0.9299092631546294
[[601  49]
 [ 41 593]]
TRAIN: [   0    1    2 ... 6417 6418 6419] TEST: [3852 3853 3854 ... 5133 5134 5135]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[623  50]
 [ 45 566]]
(1284, 1)
(1284, 1)
accuracy  0.92601246105919
precision  0.9260662132465255
recall  0.92601246105919
f1 0.9260252763982507
[[623  50]
 [ 45 566]]
TRAIN: [   0    1    2 ... 5133 5134 5135] TEST: [5136 5137 5138 ... 6417 6418 6419]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[597  64]
 [ 33 590]]
(1284, 1)
(1284, 1)
accuracy  0.9244548286604362
precision  0.9255528268067923
recall  0.9244548286604362
f1 0.9244647723900095
[[597  64]
 [ 33 590]]


In [ ]:
y_pred_LR = classifier_LR_Val.predict(X_test_3)

In [ ]:
y_pred_LR=y_pred_LR.reshape(-1,1)

In [ ]:
Y_test_3=Y_test_3.reshape(-1,1)

In [ ]:
print(y_pred.shape)
print(Y_test_3.shape)

(2140,)
(2140, 1)


In [ ]:
Evaluation_metrics(Y_test_3, y_pred_LR)

accuracy  0.9196261682242991
precision  0.9196182807734102
recall  0.9196261682242991
f1 0.9196149787137852
[[1037   83]
 [  89  931]]


SVM

In [ ]:
kf=KFold(n_splits=5, random_state=0, shuffle=False)
for train_index, test_index in kf.split(X_train_3):
  print("TRAIN:", train_index, "TEST:", test_index)
  x_train, x_test = X_train_3[train_index], X_train_3[test_index]
  y_train, y_test = Y_train_3[train_index], Y_train_3[test_index]
  y_test = y_test.reshape(-1,1)
  y_train = y_train.reshape(-1,1)
  classifier_SVC = SVC(kernel ="rbf")  
  classifier_SVC.fit(x_train, y_train)
  Y_pred = classifier_SVC.predict(x_test)
  Y_pred = Y_pred.reshape(-1,1)
  #cm = confusion_matrix(y_test,Y_pred)
  #print(cm)
  #print (Y_pred.shape)
  #print (y_test.shape)
  Evaluation_metrics(y_test, Y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


TRAIN: [1284 1285 1286 ... 6417 6418 6419] TEST: [   0    1    2 ... 1281 1282 1283]
accuracy  0.9431464174454829
precision  0.9446860349836608
recall  0.9431464174454829
f1 0.9432153504797999
[[634  54]
 [ 19 577]]
TRAIN: [   0    1    2 ... 6417 6418 6419] TEST: [1284 1285 1286 ... 2565 2566 2567]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy  0.9361370716510904
precision  0.9361827075476853
recall  0.9361370716510904
f1 0.9361507711979149
[[645  43]
 [ 39 557]]
TRAIN: [   0    1    2 ... 6417 6418 6419] TEST: [2568 2569 2570 ... 3849 3850 3851]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy  0.9400311526479751
precision  0.9403595446620961
recall  0.9400311526479751
f1 0.9400305342830625
[[603  47]
 [ 30 604]]
TRAIN: [   0    1    2 ... 6417 6418 6419] TEST: [3852 3853 3854 ... 5133 5134 5135]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy  0.9400311526479751
precision  0.9405024232105601
recall  0.9400311526479751
f1 0.9400609038867582
[[625  48]
 [ 29 582]]
TRAIN: [   0    1    2 ... 5133 5134 5135] TEST: [5136 5137 5138 ... 6417 6418 6419]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy  0.9361370716510904
precision  0.9373153214400608
recall  0.9361370716510904
f1 0.9361445091941523
[[604  57]
 [ 25 598]]
